In [28]:
'''
2023.07.22 토
이 코드파일은 크롤링된 데이터셋을 확인 후 합치고, 그 중 무료인 강의만을 선택해서 데이터셋으로 만드는 코드입니다.
나중에 코드 리팩토링을 해서 유데미의 무료강의에 대해서 처리하는 코드를 추가해 하나로 정리하면 좋을듯합니다!
참고용 카테고리 정보(인덱스 0~9)
categories = ["web-development", "data-science", "mobile-apps", "programming-languages", "game-development",
              "databases", "software-testing", "software-engineering", "development-tools", "no-code-development"]

'''

# 1. 필요 모듈 가져오기
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
import datetime
import time
import pandas as pd
import numpy as np
import pickle as pkl
from tqdm import tqdm

In [29]:
df0 = pd.read_csv('../재현/udemy_csv/test_udemy_web-development.csv')
df2 = pd.read_csv('../찬혁/Crawling/udemy_data/udemy_mobile-apps_230718.csv')
df1 = pd.read_csv('../찬혁/Crawling/udemy_data/udemy_data-science_230718.csv')
# df3 = 재현님 수집중
df4 = pd.read_csv('../재현/udemy_csv/test_udemy_game-development.csv')
df5 = pd.read_csv('../재현/udemy_csv/test_udemy_databases.csv')
# df6 = 재현님 확인중 pd.read_csv('../재현/udemy_csv/test_udemy_software-testing.csv')
df7 = pd.read_csv('../인호/02_Crawling/udemy_software-engineering_230720.csv')
df8 = pd.read_csv('../인호/02_Crawling/udemy_development-tools_230720.csv')
df9 = pd.read_csv('../인호/02_Crawling/udemy_no-code-development_230720.csv')

# 새로운 df 만든 후 인덱스 리셋
new_df = pd.concat([df0, df1, df2, df4, df5, df7, df8, df9], ignore_index=True)

print(len(new_df))
new_df.head()

8929


,Unnamed: 0,대분류,소분류,강의명,난이도,가격(현재가격),가격(원래가격),총소요시간,강의소개,언어,출처
0,0.0,web-development,Web Development,【한글자막】 The Web Developer 부트캠프 2023,모든 수준,"₩17,000","₩88,000",73.5시간 주문형 동영상,"배울 내용\n웹 개발 최신 문법 (HTML5, CSS3, 모던 자바스크립트)\nBo...",한국어,https://www.udemy.com/course/the-web-developer...
1,1.0,web-development,Docker,【한글자막】 Docker & Kubernetes : 실전 가이드,모든 수준,"₩17,000","₩88,000",23시간 주문형 동영상,"배울 내용\nDocker와 Kubernetes가 무엇이며, 왜 사용해야 하는지를 배...",한국어,https://www.udemy.com/course/docker-kubernetes...
2,2.0,web-development,Web Development,【한글자막】 100일 코딩 챌린지 - Web Development 부트캠프,모든 수준,"₩17,000","₩88,000",79시간 주문형 동영상,배울 내용\n웹 작동 방식과 웹 개발자로 시작하는 방법\n100일 안에 웹 개발 배...,한국어,https://www.udemy.com/course/100-2022-web-deve...
3,3.0,web-development,React JS,모던 리액트(React)와 리덕스(Redux),모든 수준,"₩17,000","₩88,000",18시간 주문형 동영상,배울 내용\n리액트와 리덕스를 가지고 멋진 싱글 페이지 어플리케이션 만듭니다\n리덕...,한국어,https://www.udemy.com/course/react-redux-korean/
4,4.0,web-development,JavaScript,클린코드 자바스크립트 (2023 업데이트),중급자,"₩17,000","₩88,000",10.5시간 주문형 동영상,배울 내용\n자바스크립트로 코드를 작성하는 방법에 대해 고민해봅니다.\n코드를 작성...,한국어,https://www.udemy.com/course/clean-code-js/


In [30]:
##가격(현재가격)이 "무료"인것만
free_df = new_df[new_df["가격(현재가격)"]=="무료"]
print(len(free_df), "개의 무료 강의가 있습니다")

1048 개의 무료 강의가 있습니다


In [31]:
free_df = free_df.drop(columns=['Unnamed: 0'])
free_df = free_df.reset_index(drop=True)
free_df.head()

,대분류,소분류,강의명,난이도,가격(현재가격),가격(원래가격),총소요시간,강의소개,언어,출처
0,web-development,HTML,[생활코딩] WEB - HTML & Internet 기초 #1,초급자,무료,NaN,NaN,NaN,한국어,https://www.udemy.com/course/web-html-internet-1/
1,web-development,Docker,[생활코딩] 도커(Docker) 기초,초급자,무료,NaN,NaN,NaN,한국어,https://www.udemy.com/course/docker-ky/
2,web-development,Django,[생활코딩] WEB - Python - Django Web Framework,초급자,무료,NaN,NaN,NaN,한국어,https://www.udemy.com/course/web-python-django...
3,web-development,Flask,[생활코딩] WEB - Python - Flask Web Framework,초급자,무료,NaN,NaN,NaN,한국어,https://www.udemy.com/course/web-python-flask-...
4,web-development,HTML,[생활코딩] WEB - HTML & Internet 기초 #2,초급자,무료,NaN,NaN,NaN,한국어,https://www.udemy.com/course/web-html-internet-2/


In [32]:
# 태그가 없는 경우, nan값을 할당하고 넘어가는 함수를 미리 정의합니다.
def find_element_nan(driver, path):
    try:
        element = driver.find_element(By.XPATH, path)
        return element.text
    except:
        return np.nan

In [41]:
# 🌟 중간에 코드가 끊겨서 다시 재시작한다면 restart = True 로 변경
restart = False

# 1. 실행시간 측정
start_time = time.time()


# 2. free_current_info를 저장할 pkl 파일 정보 확인
free_current_info = {"idx": 150} #원래 0인데 150쯤에 끊겨서 재실행!

if restart == True :
    free_df = pd.read_csv(f"./udemy_free_230722.csv") 
    with open('free_current_info.pickle', 'rb') as file:
        free_current_info = pkl.load(file)
    idx = free_current_info['idx']

idx = free_current_info['idx']

# 3. 크롤링 시작
print(f"크롤링 시작 : 총 {len(free_df)}페이지 중 {idx}번째 강의부터 수집 시작 ")

try:
    options = uc.ChromeOptions()
    driver = uc.Chrome(use_subprocess=True, options=options)
    for index, row in tqdm(free_df.iloc[idx:].iterrows()):
        # 4. pickle 파일 저장
        free_current_info["idx"] = index
        with open('free_current_info.pickle', 'wb') as file: 
            pkl.dump(free_current_info, file)
        # 5. row를 하나씩 돌면서 ['출처']에 접속한 후 총소요시간, 강의소개 채우기!
        url = row['출처']
        while True:
            try:     
                driver.get(url)
                break
            except Exception as e:
                exception_name = type(e).__name__
                print(f"🥲 네트워크가 연결이 잘 안돼요... Exception: {exception_name}")
                time.sleep(3)
        driver.maximize_window()
        time.sleep(3)
        free_df.at[index, '총소요시간'] = find_element_nan(driver, '//div[@class="course-content-length--course-content-length--zNAIv"]')
        free_df.at[index, '강의소개'] = find_element_nan(driver, '//div[@data-purpose="content-container-wrapper"]')
        free_df.to_csv(f"./udemy_free_230722.csv", index=False)
    time.sleep(3)
    driver.quit()    
    print(f"👍 크롤링 완료 : 총 {len(free_df)}페이지 수집 완료")

except KeyboardInterrupt:
    print(f"👍 크롤링 완료 : 총 {len(free_df)}페이지 중 {idx}번째 강의까지 수집 완료")

크롤링 시작 : 총 1048페이지 중 150번째 강의부터 수집 시작 


898it [1:08:26,  4.57s/it]


👍 크롤링 완료 : 총 1048페이지 수집 완료


In [34]:
# # test!!
# free_df_sample = free_df[:3]

# options = uc.ChromeOptions()
# driver = uc.Chrome(use_subprocess=True, options=options)
# idx=1
# for index, row in free_df_sample.iloc[idx:].iterrows():
#     url = row['출처']
#     driver.get(url)
#     free_df_sample.at[index, '총소요시간'] = find_element_nan(driver, '//div[@class="course-content-length--course-content-length--zNAIv"]')
#     free_df_sample.at[index, '강의소개'] = find_element_nan(driver, '//div[@data-purpose="content-container-wrapper"]')
# time.sleep(3)
# driver.quit

# print(free_df_sample)

               대분류     소분류                                         강의명  난이도  \
0  web-development    HTML          [생활코딩] WEB - HTML & Internet 기초 #1  초급자   
1  web-development  Docker                        [생활코딩] 도커(Docker) 기초  초급자   
2  web-development  Django  [생활코딩] WEB - Python - Django Web Framework  초급자   

  가격(현재가격) 가격(원래가격)            총소요시간  \
0       무료      NaN              NaN   
1       무료      NaN     온디맨드 동영상 42분   
2       무료      NaN  온디맨드 동영상 1시간 9분   

                                                강의소개   언어  \
0                                                NaN  한국어   
1       도커 기본개념 & 설치하기\n도커 이미지pull\n컨테이너run\n도커 네트워크  한국어   
2  Django(장고) 설치 방법\nDjango(장고) 기본 개념\nDjango(장고)...  한국어   

                                                  출처  
0  https://www.udemy.com/course/web-html-internet-1/  
1            https://www.udemy.com/course/docker-ky/  
2  https://www.udemy.com/course/web-python-django...  
